### 1. Import Image Dataset
from file `ISIC_2017_Data_GroundTruth_Classification.csv`, get the value of `melanoma` corresponding for each `image_id` inside directory `PROJECT_Data` only

In [1]:
import pandas as pd


df = pd.read_csv('ISIC_2017_Data_GroundTruth_Classification.csv')
df.head()

,image_id,melanoma,seborrheic_keratosis
0,ISIC_0000000,0.0,0.0
1,ISIC_0000001,0.0,0.0
2,ISIC_0000002,1.0,0.0
3,ISIC_0000003,0.0,0.0
4,ISIC_0000004,1.0,0.0


In [5]:
import feature_extraction as bjir
import os

dataset_path = 'Z:/per-coolyeah-an/pengolahan-citra-dan-visi-komputer/Image_Processing/uas-pcvk/PROJECT_Data' # enih pathnya diganti

ids = sorted(list(set([filename[:12] for filename in os.listdir(dataset_path)]))) # ngok
len(ids)

200

### 2. Extract Features and Save as CSV
calculate value of:
1. asymmetry
2. border_irregularity
3. color_irregularity
4. differential_structure
   
for each `image_id` inside `PROJECT_Data`, then save as `dataset.csv` 

In [6]:
features = {
    'image_id': [],
    'melanoma': [],
    'asymmetry': [],
    'border_irregularity': [],
    'color_irregularity': [],
    'differential_structure': []
}

for id in ids:
    img_features = bjir.get_features(dataset_path, id)
    img_features['image_id'] = id
    img_features['melanoma'] = int(df[df['image_id'] == id].iloc[0]['melanoma'])
    for feature_name in features:
        features[feature_name].append(img_features[feature_name])

fd = pd.DataFrame(features)

In [7]:
fd.head(10)

,image_id,melanoma,asymmetry,border_irregularity,color_irregularity,differential_structure
0,ISIC_0000000,0,0.004596,1.376352,39.668585,0.000033
1,ISIC_0000001,0,0.001275,1.617571,26.345351,0.000222
2,ISIC_0000002,1,0.002081,1.958622,27.968183,0.000050
3,ISIC_0000003,0,0.004378,1.515998,33.766395,0.000046
4,ISIC_0000004,1,0.005216,1.270549,34.367256,0.000044
5,ISIC_0000006,0,0.005461,1.841533,25.227694,0.000171
6,ISIC_0000007,0,0.001515,1.394911,25.897160,0.000176
7,ISIC_0000008,0,0.001178,1.652024,33.650311,0.000046
8,ISIC_0000009,0,0.004175,1.713577,25.223572,0.000067
9,ISIC_0000010,0,0.001681,1.591705,27.653117,0.000191


In [8]:
fd.describe()
fd.to_csv('dataset.csv')

### 3. Train AI Classifier
using `dataset.csv`, build an AI Classifier to predict `melanoma` (either 1 or 0) using value of 
1. asymmetry
2. border_irregularity
3. color_irregularity
4. differential_structure

#### 3.1 Machine Learning
Logistic Regression, SVM, KNN, LDA

In [20]:
# NOTE: mending di run di google.colab, upload 'dataset.csv' yg udah diekstrak

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [15]:
data = pd.read_csv('dataset.csv')

X = data.drop(['melanoma', 'image_id'], axis=1)
y = data['melanoma']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print(classification_report(y_test, y_pred))
joblib.dump(model, "models/logistic_regression.joblib")

Accuracy: 0.90
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        21
           1       0.89      0.89      0.89        19

    accuracy                           0.90        40
   macro avg       0.90      0.90      0.90        40
weighted avg       0.90      0.90      0.90        40



In [17]:
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=1.0)

svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred))

Accuracy: 0.93
              precision    recall  f1-score   support

           0       0.91      0.95      0.93        21
           1       0.94      0.89      0.92        19

    accuracy                           0.93        40
   macro avg       0.93      0.92      0.92        40
weighted avg       0.93      0.93      0.92        40



In [18]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=20)
knn_model.fit(X_train, y_train)

y_pred = knn_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred))

Accuracy: 0.95
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        21
           1       1.00      0.89      0.94        19

    accuracy                           0.95        40
   macro avg       0.96      0.95      0.95        40
weighted avg       0.95      0.95      0.95        40



In [19]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X_train, y_train)

y_pred = knn_model.predict(X_test)
test_accuracy = lda_model.score(X_test, y_test)
print("Test accuracy:", test_accuracy)
print(classification_report(y_test, y_pred))

Test accuracy: 0.925
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        21
           1       1.00      0.89      0.94        19

    accuracy                           0.95        40
   macro avg       0.96      0.95      0.95        40
weighted avg       0.95      0.95      0.95        40



#### 3.2 Neural Network
(akurasinya masi rendah coi :"v)

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

data = pd.read_csv('dataset.csv')
X = data[['asymmetry', 'border_irregularity', 'color_irregularity', 'differential_structure']]
y = data['melanoma']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), 
              loss=BinaryCrossentropy(), 
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=20, validation_split=0.2)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int) # probabilitas -> klasifikasi

print(classification_report(y_test, y_pred))

Epoch 1/50


C:\Users\Azka\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.6705 - loss: 0.6659 - val_accuracy: 0.7500 - val_loss: 0.6359
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6475 - loss: 0.6695 - val_accuracy: 0.7188 - val_loss: 0.6333
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6488 - loss: 0.6791 - val_accuracy: 0.7500 - val_loss: 0.6319
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5841 - loss: 0.6809 - val_accuracy: 0.7812 - val_loss: 0.6300
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7207 - loss: 0.6457 - val_accuracy: 0.7812 - val_loss: 0.6268
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6617 - loss: 0.6485 - val_accuracy: 0.7812 - val_loss: 0.6245
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6586 - loss: 0.6424 - val_accuracy: 0.7812 - val_loss: 0.6222
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6982 - loss: 0.6390 - val_accuracy: 0.7812 - val_loss: 0.6194
Epoch 9/50
7/7 ━━━

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Load dataset from CSV file
df = pd.read_csv('dataset.csv')

# Separate features (X) and target variable (y)
X = df[['asymmetry', 'border_irregularity', 'color_irregularity', 'differential_structure']].values
y = df['melanoma'].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(4,)),  # 4 input features
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss=BinaryCrossentropy(), 
              metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test),
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {accuracy}")

# Predict on the test set
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)
print(classification_report(y_test, y_pred))

Epoch 1/50


C:\Users\Azka\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.4917 - loss: 0.7635 - val_accuracy: 0.5250 - val_loss: 0.7034
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4626 - loss: 0.7808 - val_accuracy: 0.5500 - val_loss: 0.6945
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5072 - loss: 0.7096 - val_accuracy: 0.5500 - val_loss: 0.6879
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5484 - loss: 0.7087 - val_accuracy: 0.5500 - val_loss: 0.6843
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5609 - loss: 0.7042 - val_accuracy: 0.5000 - val_loss: 0.6802
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5335 - loss: 0.7111 - val_accuracy: 0.4750 - val_loss: 0.6769
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5195 - loss: 0.7071 - val_accuracy: 0.5250 - val_loss: 0.6736
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5275 - loss: 0.7167 - val_accuracy: 0.5500 - val_loss: 0.6706
Epoch 9/50
5/5 ━